In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import radians, cos, sin, asin, sqrt
from sklearn.svm import OneClassSVM
#from functionsPredictions import *
from sklearn import preprocessing
import matplotlib.pyplot as plt
import category_encoders as ce
from datetime import timedelta
from scipy import stats
import lightgbm as lgb
import seaborn as sns
import xgboost as xgb
import datetime as dt
import pandas as pd
import numpy as np
import warnings
import glob




# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_holiday2)

In [2]:
def add_time_vars(data, onehot=True):
    '''
    Adds columns for the month and weekday, and also the one-hot encoding or the cyclical versions of those features.

    :data: Dataframe that contains the a column with the datetime
    :onehot: Use onehot encoding if true and cyclical features if false (default = True)
    
    Returns a Dataframe with either the one-hot encoding or the sine and cosine of the month, weekday and time added
    '''
    data = data.reset_index()
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    if onehot == True:
        data['Year'] = pd.Categorical(data['Datetime'].dt.year)
        data['Month'] = pd.Categorical(data['Datetime'].dt.month)
        data['Weekday'] = pd.Categorical(data['Datetime'].dt.weekday)
        data['Hour'] =  pd.Categorical(data['Datetime'].dt.hour)
        data['Minute'] =  pd.Categorical(data['Datetime'].dt.minute)

        year_dummies = pd.get_dummies(data[['Year']], prefix='Year_')
        month_dummies = pd.get_dummies(data[['Month']], prefix='Month_')
        weekday_dummies = pd.get_dummies(data[['Weekday']], prefix='Weekday_')
        hour_dummies = pd.get_dummies(data[['Hour']], prefix='Hour_')
        minute_dummies = pd.get_dummies(data[['Minute']], prefix='Minute_')
        
        data = data.merge(year_dummies, left_index = True, right_index = True)
        data = data.merge(month_dummies, left_index = True, right_index = True)
        data = data.merge(weekday_dummies, left_index = True, right_index = True)
        data = data.merge(hour_dummies, left_index = True, right_index = True)
        data = data.merge(minute_dummies, left_index = True, right_index = True)
        
    data = data.set_index('Datetime')
    return data


In [3]:
df_lgbm = pd.read_csv("Merged_data.csv")
df_lgbm['Visits'] = df_lgbm['Visits'].astype('int')

In [6]:
df_lgbm = df_lgbm[df_lgbm.Location == 'Erasmuspark']

df_lgbm[["Date", 'Time']] = df_lgbm[["Date", 'Time']].astype('str')
df_lgbm['Datetime'] = pd.to_datetime(df_lgbm.Date + ' ' + df_lgbm.Time, format='%Y/%m/%d %H:%M:%S')

df_lgbm = add_time_vars(df_lgbm)

In [15]:
X = df_lgbm.drop(columns='Visits')
y = df_lgbm['Visits']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [17]:
a = X_train.columns.tolist()[5:]

X_train = X_train[a]
X_test = X_test[a]

In [18]:
X_train.columns.tolist()

['Journeys',
 'Windspeed',
 'Temperature',
 'Clouds',
 'Rain amount',
 'Rain duration',
 'Sun duration',
 'Fog',
 'Rain',
 'Snow',
 'Thunder',
 'Ice',
 'Holiday_Count',
 'Year',
 'Month',
 'Day_x',
 'Holiday_Name_Ascension Thursday',
 'Holiday_Name_Easter',
 'Holiday_Name_Fall holiday',
 'Holiday_Name_Good Friday',
 'Holiday_Name_Kings day',
 'Holiday_Name_Liberation Day',
 'Holiday_Name_May holiday',
 'Holiday_Name_Spring holiday',
 'Holiday_Name_Summer holiday',
 'Holiday_Name_Whit',
 'Holiday_Name_Winter holiday',
 'retail_and_recreation_percent_change_from_baseline',
 'grocery_and_pharmacy_percent_change_from_baseline',
 'parks_percent_change_from_baseline',
 'transit_stations_percent_change_from_baseline',
 'workplaces_percent_change_from_baseline',
 'residential_percent_change_from_baseline',
 'stringency_index',
 'Weekday',
 'Hour',
 'Minute',
 'Year__2020',
 'Year__2021',
 'Year__2022',
 'Month__1',
 'Month__2',
 'Month__3',
 'Month__4',
 'Month__5',
 'Month__6',
 'Month__7',
 

In [ ]:
model = lgb.LGBMClassifier(learning_rate=0.9,max_depth=15,random_state=42)
model.fit(X_train,y_train)

In [ ]:
# predict the results
y_pred=model.predict(X_test)

In [ ]:
y_pred.tolist()

In [ ]:
X_test

In [5]:
df_lgbm

,Unnamed: 0,Location,Visits,Date,Time,Journeys,Windspeed,Temperature,Clouds,Rain amount,...,Holiday_Name_Summer holiday,Holiday_Name_Whit,Holiday_Name_Winter holiday,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,stringency_index
0,0,Erasmuspark,243,2020-10-01,00:15:00,85.861869,5.0,15.575,8.00,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
5,5,Erasmuspark,228,2020-10-01,00:30:00,85.829948,5.0,15.550,8.00,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
10,10,Erasmuspark,175,2020-10-01,00:45:00,85.798027,5.0,15.525,8.00,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
15,15,Erasmuspark,190,2020-10-01,01:00:00,85.766106,5.0,15.500,8.00,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
20,20,Erasmuspark,128,2020-10-01,01:15:00,0.000000,5.0,15.500,8.00,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724515,724515,Erasmuspark,134,2022-01-06,13:45:00,14.358348,4.5,5.950,2.00,0.0,...,0,0,1,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44
724535,724535,Erasmuspark,165,2022-01-06,14:00:00,14.345550,5.0,6.000,1.00,0.0,...,0,0,1,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44
724555,724555,Erasmuspark,179,2022-01-06,14:15:00,14.332752,5.0,5.800,1.75,0.0,...,0,0,1,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44
724575,724575,Erasmuspark,158,2022-01-06,14:30:00,14.319953,5.0,5.600,2.50,0.0,...,0,0,1,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44
